# Base de datos de árboles en CABA

Importamos la versión de 2011: https://data.buenosaires.gob.ar/dataset/arbolado-espacios-verdes

- [Ejercicio 1](#ej-1) - Armar dataframes para Jacarandás y Palos borrachos
- [Ejercicio 2](#ej-2) - Cálculos para cada una de las especies
- [Ejercicio 3](#ej-3) - Cantidad de árboles nativos de cada especie

## Import y estructura del dataset

In [180]:
import pandas as pd

archivo = 'Materiales/arbolado-en-espacios-verdes.csv'
df = pd.read_csv(archivo, index_col = 2)

# df.head(5) # Mostrar algunas entradas
# df.columns # Analizamos columnas, aunque mejor puede ser con df.info, para ver la cantidad de entradas no NaN
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51502 entries, 1 to 51729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   long        51502 non-null  float64
 1   lat         51502 non-null  float64
 2   altura_tot  51502 non-null  int64  
 3   diametro    51502 non-null  int64  
 4   inclinacio  51502 non-null  int64  
 5   id_especie  51502 non-null  int64  
 6   nombre_com  51502 non-null  object 
 7   nombre_cie  51502 non-null  object 
 8   tipo_folla  51502 non-null  object 
 9   espacio_ve  51502 non-null  object 
 10  ubicacion   50529 non-null  object 
 11  nombre_fam  51502 non-null  object 
 12  nombre_gen  51502 non-null  object 
 13  origen      51502 non-null  object 
 14  coord_x     51502 non-null  float64
 15  coord_y     51502 non-null  float64
dtypes: float64(4), int64(4), object(8)
memory usage: 6.7+ MB


## Ejercicio 1<a class="anchor" id='ej1'></a>
#### Armar un dataframe que contenga las filas de Jacarandás y otro con Palos borrachos

Primero vemos si hay distintas especies de cada una, o si el nombre está un poco modificado, si hay variantes, etc.

In [55]:
df['nombre_com'].unique() # Mostrar todos los nombres distintos (entre ellos, aparece Palo borracho pero no da el tamaño de jupyter para mostrarlo...)

# También podemos mostrar cuántos árboles hay de cada tipo, filtrando, pero quedándonos con el límite de jupyter (60~70)
df.groupby(by='nombre_fam')['long'].count().head(5)

nombre_fam
Aceráceas        645
Adoxaceae          1
Agavaceae          8
Anacardiáceas    173
Apocynaceae       43
Name: long, dtype: int64

Buscamos Jacarandás, que podrían estar escritos con o sin tilde.

In [178]:
# Buscar Jacarandás
df_jacarandas = df[df['nombre_com'] == 'Jacarandá'] # Esto funciona en este caso porque no hay variaciones en Jacarandás, pero lo mejor sería hacer:
df_jacarandas = df[df['nombre_com'].str.contains('Jacarandá|Jacaranda')] # El '|' funciona como OR pero al ser CONTAINS, cualquier cadena que los contenga funcionará
df_jacarandas = df[df['nombre_com'].str.contains('|'.join(['Jacarandá','Jacaranda']))] # Forma alternativa de lo anterior
df_jacarandas = df[df['nombre_com'].isin(['Jacarandá','Jacaranda'])] # Usando ISIN se busca los valores que coinciden EXACTAMENTE

print(f'Hay en total {df_jacarandas.shape[0]} entradas para los tipos de arboles {df_jacarandas['nombre_com'].unique()[0]}')
# df_jacarandas.info()

Hay en total 3255 entradas para los tipos de arboles Jacarandá


Búsqueda de palos borrachos, que en principio podrían ser simplemente 'palos borrachos', pero se encontraron dos variantes más.

In [79]:
# df_palos_borrachos = df[df['nombre_com'][:(len('Palo borracho'))] == 'Palo borracho'] # No funciona... mejor usar el contains
df_palos_borrachos = df[df['nombre_com'].str.contains('Palo borracho')]

print(f'Hay en total {df_palos_borrachos.shape[0]} entradas para los tipos de arboles {df_palos_borrachos['nombre_com'].unique()})')

print('\nResumen por tipo:')
for tipo in df_palos_borrachos['nombre_com'].unique():
    print(f'Hay en total {df_palos_borrachos[df_palos_borrachos['nombre_com'] == tipo].shape[0]} entradas para el tipo {tipo}')

# Seguimos trabajando solo con el tipo 'Palo borracho'
df_palos_borrachos = df_palos_borrachos[df_palos_borrachos['nombre_com'] == 'Palo borracho']

Hay en total 3403 entradas para los tipos de arboles ['Palo borracho rosado' 'Palo borracho' 'Palo borracho blanco'])

Resumen por tipo:
Hay en total 3150 entradas para el tipo Palo borracho rosado
Hay en total 119 entradas para el tipo Palo borracho
Hay en total 134 entradas para el tipo Palo borracho blanco


<a class="anchor" id='ej2'></a>
## Ejercicio 2

#### Calcular para cada especie seleccionada
1. Cantidad de árboles, altura máxima, mínima y promedio, diámetro máximo, mínimo y promedio
2. Definir una función cantidad_arboles(parque) que, dado el nombre de un parque, calcule la cantidad de árboles que tiene.

In [173]:
# Ejercicio 2.1

tipos_a_buscar = ['Jacarandá','Palo borracho']
temp_df = df[df['nombre_com'].isin(tipos_a_buscar)]

temp_df

res_df = pd.DataFrame(columns=['cantidad_arboles','altura_maxima','altura_minima','altura_promedio','diametro_maximo','diametro_minimo','diametro_promedio'])
for nombre in temp_df['nombre_com'].unique():
    # curr_df.reset_index() # en caso de querer acceder a la posición cero, por ejemplo
    curr_df = df[df['nombre_com'] == nombre]
    res_df.loc[nombre] = [curr_df.shape[0],
                          curr_df['altura_tot'].max(),
                          curr_df['altura_tot'].min(),
                          curr_df['altura_tot'].mean(),
                          curr_df['diametro'].max(),
                          curr_df['diametro'].min(),
                          curr_df['diametro'].mean()]

res_df

,cantidad_arboles,altura_maxima,altura_minima,altura_promedio,diametro_maximo,diametro_minimo,diametro_promedio
Jacarandá,3255.0,49.0,1.0,10.369585,159.0,1.0,28.804301
Palo borracho,119.0,33.0,1.0,9.882353,130.0,1.0,43.436975


In [174]:
# Ejercicio 2.2

def cantidad_arboles(nombre_parque):
    '''
        Descripcion: dado el nombre de un parque, devuelve la cantidad de arboles contenidos en ese espacio verde
        :param nombre_parque: str que indica el nombre de parque
        :returns: la cantidad de arboles que coinciden el nombre_parque con la columna espacio_ve de la base de datos
    '''
    return df[df['espacio_ve'] == nombre_parque].shape[0]

cantidad_arboles('SICILIA')

782

## Ejercicio 3<a class="anchor" id='ej3'></a>

#### Definir una función cantidad_nativos(parque) que calcule la cantidad de árboles nativos.

In [182]:
def cantidad_nativos(nombre_parque):
    '''
        Descripcion: dado el nombre de un parque, devuelve la cantidad de arboles nativos
        :param nombre_parque: str que indica el nombre de parque
        :returns: la cantidad de arboles que coinciden el nombre_parque con la columna espacio_ve de la base de datos
    '''
    return df[(df['origen'].str.contains('Nativo')) & (df['espacio_ve'] == nombre_parque)].shape[0]

cantidad_nativos('GENERAL PAZ')

159

In [176]:
# Otros comandos

# df.groupby(by='nombre_fam')['long'].count().head(60)